<a href="https://colab.research.google.com/github/shyamsparrow/Jan-G4---NLP-Chatbot/blob/main/Chatbot_Flask_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/AIML/Capstone/Jan-G4---NLP-Chatbot

/content/drive/MyDrive/AIML/Capstone/Jan-G4---NLP-Chatbot


In [ ]:
!git checkout main

Already on 'main'
Your branch is up to date with 'origin/main'.


In [ ]:
!git pull

remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 11 (delta 5), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (11/11), done.
From https://github.com/shyamsparrow/Jan-G4---NLP-Chatbot
   9812b94..9cd6a17  main       -> origin/main
Updating 9812b94..9cd6a17
Fast-forward
 templates/communication.png | Bin 0 -> 1382 bytes
 templates/index.html        |   2 +-
 2 files changed, 1 insertion(+), 1 deletion(-)
 create mode 100644 templates/communication.png


# Installing Libraries

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import sys  
!{sys.executable} -m pip install contractions==0.0.18
!pip install pyspellchecker
!pip install contractions
!pip install tensorflow==2.7.0 --user
!pip install flask-ngrok
!pip install Flask
quiet = True

In [ ]:
%cd /content/drive/MyDrive/AIML/Capstone/Jan-G4---NLP-Chatbot/Utilities

/content/drive/MyDrive/AIML/Capstone/Jan-G4---NLP-Chatbot/Utilities


# Importing Model and Creating Predictions

In [ ]:
warnings.filterwarnings("ignore")

import pickle
import os
import random
import numpy as np


from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences

from NLP_text_preprocess import PreProcessing
import config

text_contents=dict()


model = load_model("/content/drive/MyDrive/AIML/Capstone/Jan-G4---NLP-Chatbot/Models/SimpleTransformer_Bal")
    
def greet_res(text):
    bot_greet=['hi','hello','hola','hey','howdy']
    usr_greet=['hi','hey','hello','hola','greetings','wassup','whats up']
    for word in text.split():
        if word in usr_greet:
            return random.choice(bot_greet)

# bot response
def bot_ress(usr_input):
    warnings.filterwarnings("ignore")
    with open(r"/content/drive/MyDrive/AIML/Capstone/Jan-G4---NLP-Chatbot/Utilities/parameters.pickle", "rb") as f:
          lab_en, tokenizer_data = pickle.load(f)
    pp = PreProcessing(to_lower = config.to_lower,remove_url=config.remove_url,
                       remove_time = config.remove_time,expand_contraction =config.expand_contraction,
                       remove_special_character=config.remove_special_character,
                       remove_punctuation=config.remove_punctuation,
                       remove_whitespace=config.remove_whitespace,
                       keep_alpha_numeric = False,
                       check_spelling=config.check_spelling,
                       remove_stopword=False,
                       lemmatize_word=config.lemmatize_word)

    pre_txt = pp.preprocess(usr_input)
    tok_txt = tokenizer_data.texts_to_sequences([pre_txt])
    pad_txt = pad_sequences(tok_txt, padding='post', maxlen=200)
    prediction = model.predict(pad_txt, verbose=0)
    pred_index = np.argmax(prediction,axis=1)
    result = lab_en.inverse_transform(pred_index)[0]
    bot_res = 'The Potential accident level for this accident is {}'.format(result)
    return bot_res
    
exit_list = ['exit','break','quit','see you later','chat with you later','end the chat','bye','ok bye']

def chat_res(usr_input):
    usr_input = usr_input.lower()
    #textcon.insert(END, f'User: {usr_input}'+'\n','usr')
    if usr_input in exit_list:
        return ("Ok bye! Chat with you later")
    else:
        if greet_res(usr_input) != None:
           return greet_res(usr_input)
        else:
          return bot_ress(usr_input)

In [ ]:
%cd /content/drive/MyDrive/AIML/Capstone/Jan-G4---NLP-Chatbot

/content/drive/MyDrive/AIML/Capstone/Jan-G4---NLP-Chatbot


In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz

--2022-01-05 15:40:33--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz
Resolving bin.equinox.io (bin.equinox.io)... 18.205.222.128, 54.161.241.46, 52.202.168.65, ...
Connecting to bin.equinox.io (bin.equinox.io)|18.205.222.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13770165 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.tgz.7’

ngrok-stable-linux- 100%[===================>]  13.13M  29.2MB/s    in 0.4s    

2022-01-05 15:40:34 (29.2 MB/s) - ‘ngrok-stable-linux-amd64.tgz.7’ saved [13770165/13770165]



In [ ]:
!tar -xvf /content/drive/MyDrive/AIML/Capstone/Jan-G4---NLP-Chatbot/ngrok-stable-linux-amd64.tgz

ngrok


In [ ]:
# if token is not working then use this link and login using google or github
# https://dashboard.ngrok.com/login
# after login got to this link and find your token 'https://dashboard.ngrok.com/get-started/your-authtoken'

!./ngrok authtoken 23H0IY10fqeKMIW7kG05JhKZMae_3Zabr2iqkU9AUcZ7CrRTP

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


# Running Flask API

In [ ]:
from flask import Flask, render_template, request
from flask_ngrok import run_with_ngrok

app = Flask(__name__)
app.static_folder = 'static'
run_with_ngrok(app)

@app.route("/")
def home():
    return render_template("index.html")

@app.route("/get")
def get_bot_response():
    userText = request.args.get('msg')
    return chat_res(userText)


if __name__ == "__main__":
    app.run()
#if __name__ == '__main__':
#    app.run(host='0.0.0.0',)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://67fe-35-202-27-145.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [05/Jan/2022 15:40:46] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [05/Jan/2022 15:40:47] "GET /static/styles/style.css HTTP/1.1" 200 -
127.0.0.1 - - [05/Jan/2022 15:40:49] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [05/Jan/2022 15:40:50] "GET /static/styles/style.css HTTP/1.1" 200 -
127.0.0.1 - - [05/Jan/2022 15:40:52] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [05/Jan/2022 15:41:00] "GET /get?msg=Hello HTTP/1.1" 200 -
127.0.0.1 - - [05/Jan/2022 15:47:42] "GET /get?msg=hello HTTP/1.1" 200 -
127.0.0.1 - - [05/Jan/2022 15:47:48] "GET /get?msg=Bye HTTP/1.1" 200 -
